In [1]:
%reload_ext autoreload
%autoreload 2

from init_notebook import *
from json_config import *


In [2]:
gui = MetalGUI(design)
design.overwrite_enabled = True

## Calculate Qubit Params

In [3]:
# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance


In [4]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

### dispersive readout function

In [5]:
qubit = TransmonQubit(4.3*u.GHz, ratio = 65)
res = ReadoutResonator(5.174*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 12.7222823033304 nH


In [6]:
# raise Exception('stop!')

# Write design

Global Variables

In [7]:
design_name = 'SiQb01'
wafer_name = 'QW'
jc_offset = 0.0
jj_size_offset = -65*u.nm
Jc = 0.2*u.uA/u.um**2
jc_calc = Jc*(1+jc_offset)
overlap = 250*u.nm
dir_string = f'./{design_name}_Jc{Jc.value}_overlap{overlap.value}nm_offset{jj_size_offset.value}nm'
os.makedirs(dir_string, exist_ok=True)
jj_string = f'JJ_{design_name}_Jc{Jc.value}_offset{jc_offset}_overlap{overlap.value}nm_offset{jj_size_offset.value}nm'

In [8]:
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from components.tm.LaunchpadWirebondCustom import LaunchpadWirebondCustom


from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital


from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

## Wirebond Pads

### Qubit default param 

In [9]:
Wirebond_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='40.5um',
                           pad_width='500um',
                           pad_height='300um',
                           pad_gap='100um',
                           taper_height='100um',
                           layer = str(qubit_layer),
                           pin_space = '200um')

Wirebond_opt['pos_x'] = '-2450um'
Wirebond_opt['pos_y'] = '2100um'
Wirebond_opt['orientation'] = '270'

wb_left = LaunchpadWirebondCustom(design, 'wb_left', options = Wirebond_opt)
Wirebond_opt['pos_x'] = '2450um'
wb_right = LaunchpadWirebondCustom(design, 'wb_right', options = Wirebond_opt)
gui.rebuild()

In [10]:
wb_json_path = f'{dir_string}/Wirebond_pad {design_name}.json'
options = wb_left.options
num_wb = 2
config = dict_to_lists(options, num_wb)
save_config(config, wb_json_path)

## Tee

In [11]:
TQ_options = dict(coupling_length='400 um + 90um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '5um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '-1800um',
               pos_y = '1800um', 
               mirror = True,
               layer = 5)
tee3 = CoupledLineTee(design,'tee3',options = TQ_options)
gui.rebuild()

In [12]:
tee_json_path = f'{dir_string}/CoupledLineTee {design_name}.json'
options = tee3.default_options
num_wb = 3
config = dict_to_lists(options, num_wb)
save_config(config, tee_json_path)

In [13]:
TQ_options = dict(coupling_length='400 um + 90um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '5um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '1850um',
               pos_y = '1800um', 
               mirror = False,
               layer = 5)
tee2 = CoupledLineTee(design,'tee2',options = TQ_options)
gui.rebuild()

In [14]:
TQ_options = dict(coupling_length='400 um + 90 um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '5um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '1200um',
               pos_y = '1800um', 
               mirror = False,
               layer= 5)
tee1 = CoupledLineTee(design,'tee1',options = TQ_options)
gui.rebuild()

## Qubit

In [15]:
w = '400um'
qubit_q1 = TransmonQubit(4*u.GHz, ratio = 65)
default_options_q1 = {'cut_l': '600um+'+w+'+'+w,
 'cut_h': '1000um',
 'gap': '70um',
 'w': w,
 'l': '550um',
 'r': w+'/2',
 'cpw_l': '100um',
 'jj_gap': '3um',
 'jj_contact_size': '3um',
 'coupling_gap': '30um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '80um',
 'coupling_pad_w': '100um',
'coupling_stub_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'layer' : '5',
 'coupling_arm':'False', 
'jj_taper_r': '30um'}
qubit_q2 = TransmonQubit(3.2*u.GHz, ratio = 65)
default_options_q2 = {'cut_l': '600um+'+w+"+"+w,
 'cut_h': '1100um',
 'gap': '70um',
 'jj_gap': '3um',
 'jj_contact_size': '3um',
 'w': w,
 'l': '880um',
 'r': w+'/2',
 'cpw_l': '150um',
 'coupling_gap': '30um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '70um',
 'coupling_pad_w': '200um',
'coupling_stub_w': '70um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'coupling_gd': '650um',
 'layer' : '5',
'coupling_arm':'False', 
'jj_taper_r': '30um'}

w3 = '250um'
qubit_q3 = TransmonQubit(4.8*u.GHz, ratio = 65)
default_options_q3 = {'cut_l': '450um+'+w3+'+'+w3,
 'cut_h': '650um',
 'gap': '70um',
 'jj_gap': '3um',
 'jj_contact_size': '3um',
 'w': w3,
 'l': '510um',
 'r': w3+'/2',
 'cpw_l': '100um',
 'coupling_gap': '20um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '70um',
 'coupling_pad_w': '100um',
'coupling_stub_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'layer' : '5', 
 'coupling_arm':'False', 
'jj_taper_r': '30um'}

Optimal inductance L: 13.676453476080171 nH
Optimal inductance L: 17.095566845100272 nH
Optimal inductance L: 11.397044563400145 nH


In [16]:
default_options_q3['pos_x'] = '-500um'
default_options_q3['pos_y'] = '1000um'
default_options_q3['orientation'] = '0'
q1 = DiffTransmonRounded(design,'Q3',options = default_options_q3)

gui.rebuild()

In [17]:
qb_json_path = f'{dir_string}/DiffTransmonRounded {design_name}.json'
options = q1.default_options
num_qb = 3
config = dict_to_lists(options, num_wb)
save_config(config, qb_json_path)

In [18]:
default_options_q2['pos_x'] = '-1000um'
default_options_q2['pos_y'] = '-2000um'
default_options_q2['orientation'] = '180'
q2 = DiffTransmonRounded(design,'Q2',options = default_options_q2)
gui.rebuild()

In [19]:
default_options_q1['pos_x'] = '-900um'
default_options_q1['pos_y'] = '-700um'
default_options_q1['orientation'] = '180'
q3 = DiffTransmonRounded(design,'Q1',options = default_options_q1)
gui.rebuild()

## CPW

### Resonator default

In [20]:
res_default_ops = {'chip': 'main',
 'layer': 5,
 'pin_inputs': {'start_pin': {'component': 'Q3', 'pin': 'cpw_stub'},
  'end_pin': {'component': 'tee1', 'pin': 'second_end'}},
 'fillet': '90um',
 'lead': {'start_straight': '300 um',
  'end_straight': '200 um',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '0.009287107827128471 m-1000um',
 'trace_width': '15 um',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'pin_width': '15 um',
 'gap_width': '8.3 um',
 'trace_gap': '8.3 um',
 '_actual_length': '8.287107827128471 mm'}

In [21]:
res_1 = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)

In [22]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '90um',
                     total_length = f'{res_1.len}'+'-1000um',
                     pin_inputs = dict(start_pin = dict(component = 'Q3', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee3', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '300 um', end_straight = '200 um')
route_options['Target Frequency'] = (res_1.freq).to(u.GHz).value
res3 = RouteMeander(design,name = 'res3',options = route_options)
gui.rebuild()

In [23]:
rr_json_path = f'{dir_string}/RouteMeander {design_name}.json'
options = res3.default_options
num_qb = 3
config = dict_to_lists(options, num_wb)
save_config(config, rr_json_path)

In [24]:
res_2 = ReadoutResonator(5.5*u.GHz, short_on_one_end = False)

In [25]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '80um',
                     total_length = f'{res_2.len}'+ '-700um',
                     pin_inputs = dict(start_pin = dict(component = 'Q2', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee2', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '2900 um', end_straight = '900 um')
route_options['Target Frequency'] = (res_2.freq).to(u.GHz).value
res2 = RouteMeander(design,name = 'res2',options = route_options)
gui.rebuild()

In [26]:
res_3 = ReadoutResonator(6*u.GHz, short_on_one_end = False)

In [27]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '90um',
                     total_length = f'{res_3.len}'+ '-700um',
                     pin_inputs = dict(start_pin = dict(component = 'Q1', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee1', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '2100 um', end_straight = '400 um')
route_options['Target Frequency'] = (res_3.freq).to(u.GHz).value
res1 = RouteMeander(design,name = 'res1',options = route_options)
gui.rebuild()

In [28]:
trans_ops = trans_options

In [29]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'wb_left', pin = 'tie'),end_pin = dict(component = 'tee3', pin = 'prime_start'))
trans_ops['fillet'] = '100um'
trans1 = RouteMixed(design,name = 'trans1',options = trans_ops)
gui.rebuild()

In [30]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'tee1', pin = 'prime_start'),end_pin = dict(component = 'tee3', pin = 'prime_end'))
trans2 = RouteStraight(design,name = 'trans2',options = trans_ops)
gui.rebuild()

In [31]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'tee1', pin = 'prime_end'),end_pin = dict(component = 'tee2', pin = 'prime_start'))
trans3 = RouteStraight(design,name = 'trans3',options = trans_ops)
gui.rebuild()

09:32PM 50s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
09:32PM 50s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
09:32PM 51s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
09:32PM 51s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [32]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'wb_right', pin = 'tie'),end_pin = dict(component = 'tee2', pin = 'prime_end'))
trans_ops['fillet'] = '100um'
trans4 = RouteMixed(design,name = 'trans4',options = trans_ops)
gui.rebuild()

09:32PM 51s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
09:32PM 51s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


## Junction probe pads

In [33]:
from components.junction.bandaged_dolan import DolanJunctionBandage
from components.misc.probe_pad import ProbePad
from components.misc.text import Text_object

### Junction default param

In [34]:
maximum_jj_w = 500 #nm
small_jj_length = 20#nm

h_pocket = 300 #um
w_pocket = 800





dose_array_options = Dict(
    pos_x = '0um', 
    pos_y = '0um',
    w_pocket = f'{w_pocket:.0f} um',
    h_pocket = f'{h_pocket:.0f} um',
    w_pad = '300 um',
    h_pad = '200 um',
    pad_gap = '7 um',
    orientation = '0',
    layer = f'{qubit_layer}',
    )

text_options = Dict(
        pos_x = '0um', 
        pos_y = '0um',
        text = 'haha',
        text_height = '20 mm',
        orientation = '0',
        layer = f'{qubit_layer}',
        subtract = 'False'
        )



JJ_options = Dict(
    pos_x = '0um',
    pos_y = '0um',
    w_pad_pin = '2 um',
    maximum_jj_width = '1100nm',
    small_jj_length = '400nm',
    w_pad_u = '3 um',
    d_pin = '250 nm',
    d_u = '1050 nm',
    total_length = '1 um',
    w_top_pin = '1050 nm',
    w_top_u = '1050 nm',
    w_bot_pin = '1050 nm',
    w_bot_u = '1050 nm',
    top_bot_offset = '0 nm' ,
    jj_extra = '600nm',
    jj_gap = '0.2um',
    fillet = '30 nm',
    # jj_gap_actual = '0.2um',
    # Lj = '10',
    resolution = '5',
    bandage = 'True',
    bandage_h = '20 um',
    bandage_w = '20 um',
    bandage_layer = '299',
    bandage_uc_layer = '699',
    bandage_loc = '0.5um',
    # Jc = '0.1',
    orientation = '270',
    pin_layer = f'{junction_layer}',
    gap_layer = f'{junction_area_layer}',
    area_layer_opt = 'False',
    jj_orientation = '0',
    tapered = 'False',
    taper_r = '10um',
    uc_override_band = '1um',)

JJ_options1 = JJ_options.copy()
JJ_options1['d_pin'] = '10um'
JJ_options1['d_u'] = '12um'
JJ_options1['w_pad_pin'] = '5um'
JJ_options1['w_pad_u'] = '6um'
JJ_options1['total_length'] = '30um'
JJ_options1['jj_extra'] = '0um'
JJ_options1['small_jj_length'] = '600nm'


def junction_probe_pad(x,y,jj_w,jj_gap, name, dose_array_options = dose_array_options, JJ_options = JJ_options):

    dose_array_options['pos_x'] = x
    dose_array_options['pos_y'] = y

    JJ_options['pos_x'] = x
    JJ_options['pos_y'] = y
    JJ_options['w_bot_pin'] = jj_w
    JJ_options['jj_gap'] = jj_gap

    dose_array = ProbePad(design, name+'ProbePad', dose_array_options)
    dolan = DolanJunctionBandage(design,name+'JJ',JJ_options)
    text_options['pos_x'] = x+f'-{w_pocket/2}um'
    text_options['pos_y'] = y #+ f'-{w_pocket/2}um'
    text_options['text'] = name
    text_options['text_height'] = '50um'
    text_options['orientation'] = '0'
    Text_object(design, name+'text',text_options)




## Junction

In [35]:
import analysis.Transmon_specifications as jj

### Parameters

In [36]:


jj1_len = (jj.find_junction_area(qubit_q1.L, Jc = jc_calc)/overlap).to(u.nm)


jj2_len = (jj.find_junction_area(qubit_q2.L, Jc = jc_calc)/overlap).to(u.nm)


jj3_len = (jj.find_junction_area(qubit_q3.L, Jc = jc_calc)/overlap).to(u.nm)


In [37]:
d_pin = 15
u_pin = 0.95
JJ_options1 = JJ_options.copy()
JJ_options1['d_pin'] = f'{d_pin}um'
JJ_options1['d_u'] = f'{d_pin+2*u_pin*2}um'
JJ_options1['w_pad_pin'] = '2.5um'
JJ_options1['w_pad_u'] = '3um'
JJ_options1['total_length'] = f'{q1.options.jj_gap}+{d_pin}um+{u_pin*2}um'
JJ_options1['jj_extra'] = '0um'
JJ_options1['small_jj_length'] = '600nm'

JJ_options1['bandage_h'] = '30um'
JJ_options1['bandage_w'] = '5um'

JJ_options1['bandage_loc'] = '7um'

In [38]:
def junction_on_qubit(x,y,jj_w,jj_gap,gap, name,  JJ_options = JJ_options):
    JJ_options['pos_x'] = x
    JJ_options['pos_y'] = y
    JJ_options['w_bot_pin'] = jj_w
    JJ_options['jj_gap'] = jj_gap
    JJ_options['total_length'] = gap
    dolan = DolanJunctionBandage(design,name+'JJ',JJ_options)
    return dolan


In [39]:
jjw = '200nm'

### Junction on qubit

In [40]:
from components.misc.rounded_rectangle import rounded_rec as rec2

In [41]:
dolanq1 = junction_on_qubit(q1.get_jj_location()[0],
                  q1.get_jj_location()[1],
                  f'{jj1_len.to(u.nm).value:.2f}nm',
                  jjw,
                  q1.options['jj_gap']+'+'+JJ_options1['d_u']+'*2',
                  'q1',
                  JJ_options = JJ_options1)
dolanq2 = junction_on_qubit(q2.get_jj_location()[0],
                  q2.get_jj_location()[1],
                  f'{jj2_len.to(u.nm).value:.2f}nm',
                  jjw,
                  q2.options['jj_gap']+'+'+JJ_options1['d_u']+'*2',
                  'q2',
                  JJ_options = JJ_options1)
dolanq3 = junction_on_qubit(q3.get_jj_location()[0],
                  q3.get_jj_location()[1],
                  f'{jj3_len.to(u.nm).value:.2f}nm',
                  jjw,
                  q3.options['jj_gap']+'+'+JJ_options1['d_u']+'*2',
                  'q3',
                  JJ_options = JJ_options1)
gui.rebuild()

09:32PM 53s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
09:32PM 53s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [42]:
jj_json_path = f'{dir_string}/{jj_string}.json'
options = dolanq3.default_options
num_qb = 3
config = dict_to_lists(options, num_wb)
save_config(config, jj_json_path)

### Junction Probe Pads

In [43]:
chip_size = 6800
max_xy = chip_size/2
JJ_options['small_jj_length'] = '600nm'

junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2}um',
                   f'{jj1_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '1A',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*1.1}um',
                   f'{jj1_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '1B',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*2.2}um',
                   f'{jj2_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '2A',dose_array_options = dose_array_options, JJ_options = JJ_options1)

junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+ h_pocket*3.3}um',
                   f'{jj2_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '2B',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*4.4}um',
                   f'{jj3_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '3A',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*5.5}um',
                   f'{jj3_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '3B',dose_array_options = dose_array_options, JJ_options = JJ_options1)

In [44]:
gui.rebuild()

09:32PM 55s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
09:32PM 55s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


# Test for new class

In [45]:
# Initialize the handler


# Run simulation for Q1
components_q1 = ['Q1', 'res1', 'tee1']
hfss_handler = HFSSHandler(design, components_q1,nmodes = 2,Ljs = [f'{qubit_q1.L}'])
results_q1 = hfss_handler.run_simulation(
    qubit=qubit_q1,
    components=components_q1,
    target_freq=4.0,
    sim_name='q1_sim'
)

# Run simulation for Q2
components_q2 = ['Q2', 'res2', 'tee2'] 
results_q2 = hfss_handler.run_simulation(
    qubit=qubit_q2,
    components=components_q2,
    target_freq=3.2,
    sim_name='q2_sim'
)

INFO 09:33PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:33PM [load_ansys_project]: 	Opened Ansys App
INFO 09:33PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 09:33PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project54
INFO 09:33PM [connect_design]: No active design found (or error getting active design).
INFO 09:33PM [connect]: 	 Connected to project "Project54". No design detected
INFO 09:33PM [connect_design]: 	Opened active design
	Design:    q1_sim_hfss [Solution type: Eigenmode]
WARNING 09:33PM [connect_setup]: 	No design setup detected.
WARNING 09:33PM [connect_setup]: 	Creating eigenmode default setup.
INFO 09:33PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:33PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:33PM [analyze]: Analyzing setup Setup
10:56PM 40s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\D

Design "q1_sim_hfss" info:
	# eigenmodes    2
	# variations    1
Design "q1_sim_hfss" info:
	# eigenmodes    2
	# variations    1


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\project_info.py: 239



        energy_elec_all       = 3.55525676424551e-26
        energy_elec_substrate = 3.24143843688428e-26
        EPR of substrate = 91.2%

        energy_mag    = 4.83775068828946e-28
        energy_mag % of energy_elec_all  = 1.4%
        

Variation 0  [1/1]


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1101
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1102
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1245


...

###### Numerical diagonalization

Lj,17.1
0,3520.39
1,5432.28


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                      
17.1 0  193.29  5.56e-01
     1    0.56  4.52e-04

In [70]:
q3d_handler.renderer_q3d.options


{'Lj': '13nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'max_mesh_length_port': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'x_buffer_width_mm': 0.5,
 'y_buffer_width_mm': 0.5,
 'wb_threshold': '72um',
 'wb_offset': '0um',
 'wb_size': 1.6,
 'wb_height': 0.7,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'},
 'substrate_height': '250um',
 'gap': '20um'}

In [ ]:
# Initialize the Q3D handler


# Setup simulation for qubit and resonator
components_q1 = ['Q1', 'res1', 'tee1']

q3d_handler = Q3DHandler(design,)

q3d_handler.setup_capacitance(components_q1,substrate_height = '500um',gap = '15um')

# Run simulation and get results
results = q3d_handler.run_simulation(
    components=components_q1,
    sim_name='qubit_resonator_coupling'
)

com_error: (-2147023174, 'The RPC server is unavailable.', None, None)

In [52]:
mat = q3d_handler.q3d.sim.capacitance_matrix


In [59]:
mat.columns


Index(['ground_main_plane', 'pad_left_Q1', 'pad_right_Q1',
       'resonator_pad_0_Q1'],
      dtype='object')

In [65]:
dat = {}
for i, col in enumerate(mat.columns):
    for j, row in enumerate(mat.columns):
        if i <= j:
            dat[f'C{i+1}{j+1} (fF)'] = mat[col][row]



In [66]:
dat

{'C11 (fF)': 2021.25705,
 'C12 (fF)': -89.35163,
 'C13 (fF)': -92.51463,
 'C14 (fF)': -1645.07586,
 'C22 (fF)': 142.80311,
 'C23 (fF)': -38.21205,
 'C24 (fF)': -13.50632,
 'C33 (fF)': 133.44937,
 'C34 (fF)': -0.55404,
 'C44 (fF)': 1660.08353}

# GDS

In [46]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [47]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
junction_bandage_layer = 299    
junction_uc_layer = 699

In [48]:
a_gds.options['max_points'] = 3000
a_gds.options['fabricate'] = True

In [49]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = False
a_gds.options['no_cheese']['view_in_file']['main'][1] = False

In [50]:
a_gds.options.tolerance = '0.0000005'
a_gds.options.precision = '0.0000000005'

In [51]:
a_gds.options['negative_mask'] = dict(main= [qubit_layer])

In [52]:
a_gds.options.cheese.edge_nocheese = '150um'
a_gds.options.no_cheese.buffer = '200um'
a_gds.options.cheese.cheese_1_radius = '10um'
a_gds.options.cheese.cheese_0_x = '10um'
a_gds.options.cheese.cheese_0_y = '10um'

In [53]:
# gui.rebuild()

for i,qubit in enumerate([q1, q2, q3]):
    update_config(qb_json_path,qubit_index=i,updates = qubit.options)
for i, res in enumerate([res1,res2,res3]):
    update_config(rr_json_path,qubit_index=i,updates = res.options)
for i, tee in enumerate([tee1,tee2,tee3]):
    update_config(tee_json_path,qubit_index=i,updates = tee.options)
for i, wb in enumerate([wb_left,wb_right]):
    update_config(wb_json_path,qubit_index=i,updates = wb.options)


a_gds.export_to_gds(f'{dir_string}/SiQb03_PW_offset{jc_offset}.gds')



 /Users/wendy/_Lib/qiskit-metal/qiskit_metal/renderers/renderer_gds/gds_renderer.py: 780


 /Users/wendy/_Lib/qiskit-metal/qiskit_metal/renderers/renderer_gds/gds_renderer.py: 780
11:29AM 53s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
11:29AM 53s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
11:29AM 53s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
11:29AM 53s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
11:29AM 54s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"/Users/wendy/De

1

In [54]:
a_gds.options['check_short_segments_by_scaling_fillet'] = '1.0'